In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
mpl.style.use('bmh')

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import r2_score
from sklearn.externals import joblib

from sorder.models.linear_regression import Corpus

In [6]:
train = Corpus('/Users/dclure/Projects/sent-order/data/train.json')

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [7]:
vocab = (
    train.most_common_ngrams(1, 3000) |
    train.most_common_ngrams(2, 2000) |
    train.most_common_ngrams(3, 1000)
)

HBox(children=(IntProgress(value=0, max=884912), HTML(value='')))

HBox(children=(IntProgress(value=0, max=884912), HTML(value='')))

HBox(children=(IntProgress(value=0, max=884912), HTML(value='')))

In [8]:
dv = DictVectorizer()

In [9]:
train_x, train_y = zip(*train.xy(vocab))

HBox(children=(IntProgress(value=0, max=884912), HTML(value='')))

In [10]:
train_x = dv.fit_transform(train_x)

In [11]:
joblib.dump(dv, 'dv.bin')

['dv.bin']

In [12]:
model = LinearRegression()

In [13]:
fit = model.fit(train_x, train_y)

In [14]:
joblib.dump(fit, 'fit.bin')

['fit.bin']